In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import mode
import pandas as pd
import pickle
import csv

In [2]:
dataSize = 2985217
properties_df = pd.read_csv('properties_2016.csv')
# df = pd.read_csv('train_2016_v2.csv')

/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
categoryNames = list(properties_df)
custom = ['parcelid',
          'regionidzip', 
          'poolcnt',
          'fireplacecnt']
binary = ['fireplaceflag',
          'hashottuborspa',
          'pooltypeid10',
          'pooltypeid2',
          'pooltypeid7',
          'taxdelinquencyflag']

toDeleteCategories = ['propertycountylandusecode',
                      'taxdelinquencyyear',
                      'propertyzoningdesc',
                     'taxvaluedollarcnt','landtaxvaluedollarcnt']
categoricalFeatures = ['airconditioningtypeid',
                       'architecturalstyletypeid',
                       'buildingclasstypeid',
                       'decktypeid',
                       'heatingorsystemtypeid',
                       'propertylandusetypeid',
                       'storytypeid',
                       'typeconstructiontypeid']
nonNumerical = categoricalFeatures + toDeleteCategories + custom + binary
numericalFeatures = [key for key in categoryNames if key not in nonNumerical]
print numericalFeatures

['basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolsizesum', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt', 'assessmentyear', 'taxamount', 'censustractandblock']


### Calculate Averages/Modes by ZIP

In [4]:
propertiesGroupedByZip = properties_df.groupby('regionidzip')
# local['bedroomcnt'].mean()

In [5]:
# categByCategoryZip = {}
# for categ in categoricalFeatures:
#     categByCategoryZip[categ] = propertiesGroupedByZip[categ].agg(lambda x: mode(x)[0])
categByCategoryZip = pickle.load(open( "categByCategoryZip.p", "rb" ))

In [6]:
numerByCategoryZip = {}
for numer in numericalFeatures:
    numerByCategoryZip[numer] = propertiesGroupedByZip[numer].mean()

### Fill in Custom Missing Values

In [7]:
mostCommonZip = int(properties_df['regionidzip'].mode())
properties_df['regionidzip'] = properties_df['regionidzip'].fillna(mostCommonZip)

In [8]:
properties_df['taxdelinquencyflag'] = properties_df['taxdelinquencyflag'].fillna(0)
properties_df['taxdelinquencyflag'] = properties_df['taxdelinquencyflag'].replace("Y", 1)
print pd.Series.value_counts(properties_df['taxdelinquencyflag'])

0    2928755
1      56462
Name: taxdelinquencyflag, dtype: int64


In [9]:
properties_df['fireplaceflag'] = properties_df['fireplaceflag'].replace(True, 1)
properties_df['fireplaceflag'] = properties_df['fireplaceflag'].fillna(0)
print pd.Series.value_counts(properties_df['fireplaceflag'])

0.0    2980054
1.0       5163
Name: fireplaceflag, dtype: int64


In [10]:
properties_df['hashottuborspa'] = properties_df['hashottuborspa'].replace(True, 1)
properties_df['hashottuborspa'] = properties_df['hashottuborspa'].fillna(0)
print pd.Series.value_counts(properties_df['hashottuborspa'])

0.0    2916203
1.0      69014
Name: hashottuborspa, dtype: int64


In [11]:
for pooltype in ['pooltypeid10','pooltypeid2','pooltypeid7','poolcnt', 'fireplacecnt']:
    properties_df[pooltype] = properties_df[pooltype].fillna(0)

### Calculate Global Averages / Modes

In [12]:
numericElemsAvgsGlobal = {}
categorElemsModesGlobal = {}
for numer in numericalFeatures:
    numericElemsAvgsGlobal[numer] = pd.Series.mean(properties_df[numer])
for categ in categoricalFeatures:
    categorElemsModesGlobal[categ] = pd.Series.mode(properties_df[categ])[0]

In [34]:
pd.options.display.max_columns = 60
properties_df.head(10)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,1.0,0.0,646.883292,0.0,0.0,4.0,5.111000,3.725835,0.0,1380.630396,3263.039524,3250.544862,1178.900678,12504.545455,1388.944578,2414.339439,6037.0,0.0,3.725835,1.823517,383.769357,0.0,2.0,34144442.0,-118654084.0,85768.000000,0.0,519.71098,0.0,0.0,0.0,010D,269.0,NaN,6.037800e+07,37688.0,3101.0,550898.818765,96337.0,0.0,0.0,1.010009,0.0,1.019796,319.803397,278.296562,1982.043313,1.666667,0.0,485771.242555,9.0,2015.0,9.0,12788.477143,0,NaN,6.037800e+13
1,10759547,1.0,0.0,646.883292,0.0,0.0,4.0,5.111000,3.725835,0.0,1380.630396,3263.039524,3250.544862,1178.900678,12504.545455,1388.944578,2414.339439,6037.0,0.0,3.725835,1.823517,383.769357,0.0,2.0,34140430.0,-118625364.0,4083.000000,0.0,519.71098,0.0,0.0,0.0,0109,261.0,LCA11*,6.037800e+07,37688.0,3101.0,550898.818765,96337.0,0.0,0.0,1.010009,0.0,1.019796,319.803397,278.296562,1982.043313,1.666667,0.0,485771.242555,27516.0,2015.0,27516.0,12788.477143,0,NaN,6.037800e+13
2,10843547,1.0,0.0,646.883292,0.0,0.0,4.0,5.315937,1.964694,0.0,1380.630396,73026.000000,1337.557450,1178.900678,73026.000000,1388.944578,2414.339439,6037.0,0.0,1.964694,1.823517,383.769357,0.0,7.0,33989359.0,-118394633.0,63085.000000,0.0,519.71098,0.0,0.0,0.0,1200,47.0,LAC2,6.037703e+07,51617.0,3101.0,550498.082019,96095.0,0.0,0.0,1.010009,0.0,2.000000,319.803397,278.296562,1960.890921,1.000000,0.0,650756.000000,1413387.0,2015.0,762631.0,20800.370000,0,NaN,6.037609e+13
3,10859147,1.0,0.0,646.883292,0.0,0.0,3.0,7.000000,2.579304,0.0,1380.630396,5068.000000,1962.979610,1178.900678,5068.000000,1388.944578,2414.339439,6037.0,0.0,2.579304,1.823517,383.769357,0.0,2.0,34148863.0,-118437206.0,7521.000000,0.0,519.71098,0.0,0.0,0.0,1200,47.0,LAC2,6.037141e+07,12447.0,3101.0,27080.000000,96424.0,0.0,0.0,1.010009,0.0,1.051462,319.803397,278.296562,1948.000000,1.000000,0.0,571346.000000,1156834.0,2015.0,585488.0,14557.570000,0,NaN,6.037137e+13
4,10879947,1.0,0.0,646.883292,0.0,0.0,4.0,6.177071,2.006557,0.0,1380.630396,1776.000000,1446.123653,1178.900678,1776.000000,1388.944578,2414.339439,6037.0,0.0,2.006557,1.823517,383.769357,0.0,2.0,34194168.0,-118385816.0,8512.000000,0.0,519.71098,0.0,0.0,0.0,1210,31.0,LAM1,6.037123e+07,12447.0,3101.0,46795.000000,96450.0,0.0,0.0,1.010009,0.0,1.000000,319.803397,278.296562,1947.000000,1.000000,0.0,193796.000000,433491.0,2015.0,239695.0,5725.170000,0,NaN,6.037122e+13
5,10898347,1.0,0.0,646.883292,0.0,0.0,4.0,7.000000,2.114563,0.0,1380.630396,2400.000000,1353.918009,1178.900678,2400.000000,1388.944578,2414.339439,6037.0,0.0,2.114563,1.823517,383.769357,0.0,2.0,34171873.0,-118380906.0,2500.000000,0.0,519.71098,0.0,0.0,0.0,1210,31.0,LAC4,6.037125e+07,12447.0,3101.0,46795.000000,96446.0,0.0,0.0,1.010009,0.0,1.383755,319.803397,278.296562,1943.000000,1.000000,0.0,176383.000000,283315.0,2015.0,106932.0,3661.280000,0,NaN,6.037125e+13
6,10933547,1.0,0.0,646.883292,0.0,0.0,4.0,4.680587,2.550228,0.0,1380.630396,2111.232238,2054.404911,1178.900678,2770.730973,1388.944578,2414.339439,6037.0,0.0,2.550228,1.823517,383.769357,0.0,2.0,34131929.0,-118351474.

In [18]:
# smallTest = properties_df
# for index, row in smallTest.iterrows():
#     zipCode = row['regionidzip']
#     for numer in numericalFeatures:
#         if np.isnan(row[numer]):
#             if zipCode in numerByCategoryZip[numer] and not np.isnan(numerByCategoryZip[numer][zipCode]):
#                 smallTest.set_value(index, numer, numerByCategoryZip[numer][zipCode])
#             else:
#                 smallTest.set_value(index, numer, numericElemsAvgsGlobal[numer])
#     for categ in categoricalFeatures:
#         if np.isnan(row[categ]):
#             if zipCode in categByCategoryZip[categ] and not np.isnan(categByCategoryZip[categ][zipCode]):
#                 smallTest.set_value(index, categ, categByCategoryZip[categ][zipCode])
#             else:
#                 smallTest.set_value(index, categ, categByCategoryZip[categ])
properties_df = pickle.load(open("almostProcessedData.p" , "rb"))

In [41]:
currentlyWeird = ['taxvaluedollarcnt','landtaxvaluedollarcnt']
for stuff in currentlyWeird:
    del properties_df[stuff]
for toDelete in toDeleteCategories:
    del properties_df[toDelete]

In [62]:
properties_df.head(10)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,assessmentyear,taxamount,taxdelinquencyflag,censustractandblock
0,10754147,1.0,0.0,646.883292,0.0,0.0,4.0,5.111000,3.725835,0.0,1380.630396,3263.039524,3250.544862,1178.900678,12504.545455,1388.944578,2414.339439,6037.0,0.0,3.725835,1.823517,383.769357,0.0,2.0,34144442.0,-118654084.0,85768.000000,0.0,519.71098,0.0,0.0,0.0,269.0,6.037800e+07,37688.0,3101.0,550898.818765,96337.0,0.0,0.0,1.010009,0.0,1.019796,319.803397,278.296562,1982.043313,1.666667,0.0,485771.242555,2015.0,12788.477143,0,6.037800e+13
1,10759547,1.0,0.0,646.883292,0.0,0.0,4.0,5.111000,3.725835,0.0,1380.630396,3263.039524,3250.544862,1178.900678,12504.545455,1388.944578,2414.339439,6037.0,0.0,3.725835,1.823517,383.769357,0.0,2.0,34140430.0,-118625364.0,4083.000000,0.0,519.71098,0.0,0.0,0.0,261.0,6.037800e+07,37688.0,3101.0,550898.818765,96337.0,0.0,0.0,1.010009,0.0,1.019796,319.803397,278.296562,1982.043313,1.666667,0.0,485771.242555,2015.0,12788.477143,0,6.037800e+13
2,10843547,1.0,0.0,646.883292,0.0,0.0,4.0,5.315937,1.964694,0.0,1380.630396,73026.000000,1337.557450,1178.900678,73026.000000,1388.944578,2414.339439,6037.0,0.0,1.964694,1.823517,383.769357,0.0,7.0,33989359.0,-118394633.0,63085.000000,0.0,519.71098,0.0,0.0,0.0,47.0,6.037703e+07,51617.0,3101.0,550498.082019,96095.0,0.0,0.0,1.010009,0.0,2.000000,319.803397,278.296562,1960.890921,1.000000,0.0,650756.000000,2015.0,20800.370000,0,6.037609e+13
3,10859147,1.0,0.0,646.883292,0.0,0.0,3.0,7.000000,2.579304,0.0,1380.630396,5068.000000,1962.979610,1178.900678,5068.000000,1388.944578,2414.339439,6037.0,0.0,2.579304,1.823517,383.769357,0.0,2.0,34148863.0,-118437206.0,7521.000000,0.0,519.71098,0.0,0.0,0.0,47.0,6.037141e+07,12447.0,3101.0,27080.000000,96424.0,0.0,0.0,1.010009,0.0,1.051462,319.803397,278.296562,1948.000000,1.000000,0.0,571346.000000,2015.0,14557.570000,0,6.037137e+13
4,10879947,1.0,0.0,646.883292,0.0,0.0,4.0,6.177071,2.006557,0.0,1380.630396,1776.000000,1446.123653,1178.900678,1776.000000,1388.944578,2414.339439,6037.0,0.0,2.006557,1.823517,383.769357,0.0,2.0,34194168.0,-118385816.0,8512.000000,0.0,519.71098,0.0,0.0,0.0,31.0,6.037123e+07,12447.0,3101.0,46795.000000,96450.0,0.0,0.0,1.010009,0.0,1.000000,319.803397,278.296562,1947.000000,1.000000,0.0,193796.000000,2015.0,5725.170000,0,6.037122e+13
5,10898347,1.0,0.0,646.883292,0.0,0.0,4.0,7.000000,2.114563,0.0,1380.630396,2400.000000,1353.918009,1178.900678,2400.000000,1388.944578,2414.339439,6037.0,0.0,2.114563,1.823517,383.769357,0.0,2.0,34171873.0,-118380906.0,2500.000000,0.0,519.71098,0.0,0.0,0.0,31.0,6.037125e+07,12447.0,3101.0,46795.000000,96446.0,0.0,0.0,1.010009,0.0,1.383755,319.803397,278.296562,1943.000000,1.000000,0.0,176383.000000,2015.0,3661.280000,0,6.037125e+13
6,10933547,1.0,0.0,646.883292,0.0,0.0,4.0,4.680587,2.550228,0.0,1380.630396,2111.232238,2054.404911,1178.900678,2770.730973,1388.944578,2414.339439,6037.0,0.0,2.550228,1.823517,383.769357,0.0,2.0,34131929.0,-118351474.0,42841.893671,0.0,519.71098,0.0,0.0,0.0,260.0,6.037144e+07,12447.0,3101.0,274049.000000,96049.0,0.0,0.0,1.010009,0.0,1.139090,319.803397,278.296562,1951.295547,1.000000,0.0,397945.000000,2015.0,6773.340000,0,6.037181e+13
7,10940747,1.0,0.0,646.883292,0.0,0.0,3.0,4.898810,2.290650,0.0,1

In [61]:
for categ in categoricalFeatures:
#     properties_df[categ] = properties_df[categ].astype('str')
    properties_df[categ] = properties_df[categ].astype(float)
    break;

In [65]:
listData = properties_df.values.tolist()

In [67]:
finalCategoryNames = list(properties_df)

In [98]:
numericalIndices = [finalCategoryNames.index(numer) for numer in numericalFeatures]
categIndices = [finalCategoryNames.index(categ) for categ in categoricalFeatures]
everythingElse = custom + binary
otherIndices = [finalCategoryNames.index(other) for other in everythingElse]

In [75]:
numerData = np.array(listData)[:,numericalIndices]

In [76]:
categData = np.array(listData)[:,categIndices]

In [99]:
everythingElseData = np.array(listData)[:,otherIndices]

### Normalize Numerical Features

In [86]:
scaler = StandardScaler().fit(numerData)
numerDataNormalized = scaler.transform(numerData)

### One-hot Encode Categorical Features

In [ ]:
enc = OneHotEncoder()
enc.fit(categData)
categDataOneHot = enc.transform(categData).toarray()

In [101]:
fullFinalData = np.hstack((everythingElseData, numerDataNormalized, categDataOneHot))

In [104]:
np.save("fullFinalData.npy",fullFinalData)

In [106]:
print(fullFinalData)

[[  1.07541470e+07   9.63370000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.07595470e+07   9.63370000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.08435470e+07   9.60950000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  1.68040630e+08   9.69870000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.68040830e+08   9.69870000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.68040430e+08   9.69870000e+04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [109]:
transactionByProperty = {}
with open('train_2016_v2.csv', 'rb') as csvfile:
    transactionReader = csv.reader(csvfile, delimiter=',')
    length = 0
    firstLine = 0
    
    for row in transactionReader:
        if firstLine == 0:
            firstLine = 1
            continue
        parcelid, logerror, transactiondate = row
        if parcelid in transactionByProperty:
            continue
        else:
            transactionByProperty[parcelid] = [(transactiondate, logerror)]
        length +=1
    print "transactions length = " + str(length)

transactions length = 90150


In [111]:
dictCache = {}
for rowNumber in range(2985217):
    dictCache[fullFinalData[rowNumber][0]] = rowNumber

In [125]:
trainingDataRowIndices = []
trainingDataPrices = []
for parcelid in transactionByProperty:
    trainingDataRowIndices.append(dictCache[float(parcelid)])
    trainingDataPrices.append([transactionByProperty[parcelid][0][1]])

In [126]:
len(trainingDataRowIndices)

90150

In [128]:
trainingDataRows = fullFinalData[trainingDataRowIndices]
prices = np.array(trainingDataPrices)

In [139]:
trainingData = np.hstack((trainingDataRows, prices))[:,1:].astype(float)

In [141]:
# np.save("trainingData.npy", trainingData)

### Clearing Out Unnecessary Columns (Too Sparse)

In [2]:
trainingData = np.load("trainingData.npy")

In [4]:
prices = trainingData[:, -1:]

In [8]:
unique = np.unique(prices)

In [9]:
unique.shape

(1892,)

In [20]:
for i in range(len(trainingData[0])):
    unique = np.unique(trainingData[:, i],  return_counts=True)
    if len(unique[0]) < 30:
        print str(i) + "th column "
        print unique
        print 

1th column 
(array([ 0.,  1.]), array([72274, 17876]))

2th column 
(array([ 0.,  1.,  2.,  3.,  4.,  5.]), array([80553,  8157,  1104,   312,    21,     3]))

3th column 
(array([ 0.,  1.]), array([89930,   220]))

4th column 
(array([ 0.,  1.]), array([87787,  2363]))

5th column 
(array([ 0.,  1.]), array([88991,  1159]))

6th column 
(array([ 0.,  1.]), array([88946,  1204]))

7th column 
(array([ 0.,  1.]), array([73478, 16672]))

8th column 
(array([ 0.,  1.]), array([88375,  1775]))

10th column 
(array([ -2.0545827 ,  -1.12516705,  -0.66045923,  -0.19575141,
         0.26895641,   0.73366423,   1.19837205,   1.66307987,
         2.12778769,   2.59249552,   3.05720334,   3.52191116,
         3.98661898,   4.4513268 ,   4.91603462,   5.38074244,
         5.84545026,   6.31015808,   7.23957373,   8.16898937,
         9.09840501,  11.88665194,  16.53373015]), array([ 1159, 15014,  1639, 36485,  8084, 19920,  1089,  3737,   793,
        1139,   253,   447,    54,   155,     9,   114

In [22]:
UselessColumns = [3, 4, 5, 6, 8, 29, 43, 48, 49, 50, 51, 52, 55, 56, 57, 59, 60, 61, 62,64, 65, 68, 72,76, 77, 78, 79 ,80, 81, 82, 83, 84, 86,87,88,89,90, 91,93,94,95,97,98,99,100,104,106,107,108]

In [25]:
uselessColumns = np.array(UselessColumns)

In [27]:
purged = np.delete(trainingData, UselessColumns, 1)

In [31]:
len(purged[:, -1:])

90150

In [32]:
np.save("trainingDataNotSpare.npy", purged)

In [33]:
fullFinalData = np.load("fullFinalData.npy")

In [34]:
purgedFinalData = np.delete(fullFinalData, UselessColumns, 1)

In [35]:
np.save("fullFinalDataNotSpare.npy", purgedFinalData)

In [42]:
np.unique(purged[:,1], return_counts=True)

(array([ 0.,  1.]), array([72274, 17876]))

### Idea: Finite types of labels -> Classification Problem

In [57]:
trainingData = np.load("../data/trainBest.npy")
validationData = np.load("../data/validBest.npy")

In [66]:
fullData = np.vstack((trainingData, validationData))
classes = np.unique(fullData[:,-1]).tolist()
classes.sort()

In [85]:
len(classes)

2780

In [72]:
# classes = train_2016_df["logerror"].unique().tolist()
# classes.sort()

1894

In [75]:
classIndexEncoding = {}
for index in range(len(classes)):
    classIndexEncoding[classes[index]] = index

In [76]:
for rowNum in range(len(validationData)):
    label = validationData[rowNum][-1]
    if label not in classIndexEncoding:
        print rowNum
        print label
        print "*************************************"
    else:
        validationData[rowNum][-1] = int(classIndexEncoding[label])

In [79]:
for rowNum in range(len(trainingData)):
    label = trainingData[rowNum][-1]
    if label not in classIndexEncoding:
        print rowNum
        print label
        print "*************************************"
    else:
        trainingData[rowNum][-1] = int(classIndexEncoding[label])

In [81]:
np.save("../data/validationDataClassify.npy", validationData)
np.save("../data/trainingDataClassify.npy", trainingData)

In [84]:
len(validationData[0])

61